In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6379,2020-12-23T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1028,176,1204,32410,...,2213,NaN,NaN,87024,1164552,779017.0,NaN,6.0,NaN,NaN
6380,2020-12-23T17:00:00,ITA,9,Toscana,43.769231,11.255889,903,166,1069,10288,...,3517,NaN,NaN,116979,1818397,1039802.0,NaN,7.0,Positivi diagnosticati solo con test antigenic...,NaN
6381,2020-12-23T17:00:00,ITA,10,Umbria,43.106758,12.388247,239,40,279,3269,...,584,NaN,NaN,27553,484909,246207.0,NaN,1.0,NaN,NaN
6382,2020-12-23T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,84,4,88,342,...,370,NaN,NaN,7084,60760,36709.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6379,2020-12-23T17:00:00,19,Sicilia,1028,176,1204,32410,33614,122,932,51197,2213,NaN,NaN,87024,1164552,779017.0,6.0,NaN,NaN
6380,2020-12-23T17:00:00,9,Toscana,903,166,1069,10288,11357,-480,435,102105,3517,NaN,NaN,116979,1818397,1039802.0,7.0,Positivi diagnosticati solo con test antigenic...,NaN
6381,2020-12-23T17:00:00,10,Umbria,239,40,279,3269,3548,-181,157,23421,584,NaN,NaN,27553,484909,246207.0,1.0,NaN,NaN
6382,2020-12-23T17:00:00,2,Valle d'Aosta,84,4,88,342,430,-26,11,6284,370,NaN,NaN,7084,60760,36709.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-23', '2020-12-22')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-23T17:00:00,1,Piemonte,3239,215,3454,33107,36561,-348,933,147203,7679,NaN,NaN,191443,1633219,966374.0,13.0,NaN,NaN
1,2020-12-23T17:00:00,2,Valle d'Aosta,84,4,88,342,430,-26,11,6284,370,NaN,NaN,7084,60760,36709.0,0.0,NaN,NaN
2,2020-12-23T17:00:00,3,Lombardia,4243,536,4779,55460,60239,-905,2153,379750,24610,NaN,NaN,464599,4705264,2556373.0,31.0,NaN,NaN
3,2020-12-23T17:00:00,5,Veneto,2660,350,3010,100316,103326,748,3357,116872,5747,NaN,NaN,225945,3165214,1177012.0,36.0,NaN,NaN
4,2020-12-23T17:00:00,6,Friuli Venezia Giulia,595,56,651,11624,12275,-941,512,32110,1499,NaN,NaN,45884,887279,330123.0,6.0,"Su indicazione del Ministero della Salute, il ...",NaN
5,2020-12-23T17:00:00,7,Liguria,689,64,753,5424,6177,-291,228,49202,2819,NaN,NaN,58198,684768,315604.0,5.0,I 139001 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-23T17:00:00,8,Emilia-Romagna,2803,207,3010,55015,58025,-1721,1129,94188,7259,NaN,NaN,159472,2469690,1194299.0,21.0,NaN,NaN
7,2020-12-23T17:00:00,9,Toscana,903,166,1069,10288,11357,-480,435,102105,3517,NaN,NaN,116979,1818397,1039802.0,7.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-23T17:00:00,10,Umbria,239,40,279,3269,3548,-181,157,23421,584,NaN,NaN,27553,484909,246207.0,1.0,NaN,NaN
9,2020-12-23T17:00:00,11,Marche,411,61,472,9087,9559,62,498,27238,1495,NaN,NaN,38292,499038,294678.0,3.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-22T17:00:00,1,Piemonte,3331,228,3559,33350,36909,-1048,542,145980,7621,NaN,NaN,190510,1624273,963280.0,7.0,NaN,NaN
1,2020-12-22T17:00:00,2,Valle d'Aosta,90,5,95,361,456,-18,33,6249,368,NaN,NaN,7073,60537,36564.0,1.0,NaN,NaN
2,2020-12-22T17:00:00,3,Lombardia,4290,540,4830,56314,61144,-170,2278,376790,24512,NaN,NaN,462446,4672338,2546141.0,23.0,NaN,NaN
3,2020-12-22T17:00:00,5,Veneto,2672,349,3021,99557,102578,1104,3082,114379,5631,NaN,NaN,222588,3144621,1171789.0,40.0,NaN,NaN
4,2020-12-22T17:00:00,6,Friuli Venezia Giulia,598,57,655,12561,13216,-31,549,31291,1477,NaN,NaN,45984,880631,328586.0,7.0,NaN,NaN
5,2020-12-22T17:00:00,7,Liguria,700,65,765,5703,6468,-120,216,48704,2798,NaN,NaN,57970,680832,314620.0,1.0,I 135297 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-22T17:00:00,8,Emilia-Romagna,2891,210,3101,56645,59746,-2308,1162,91411,7188,NaN,NaN,158345,2451777,1189534.0,24.0,NaN,NaN
7,2020-12-22T17:00:00,9,Toscana,941,175,1116,10721,11837,-559,309,101212,3495,NaN,NaN,116544,1806133,1035769.0,3.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-22T17:00:00,10,Umbria,244,44,288,3441,3729,-124,114,23089,578,NaN,NaN,27396,481344,245205.0,5.0,NaN,NaN
9,2020-12-22T17:00:00,11,Marche,421,61,482,9015,9497,-100,363,26806,1491,NaN,NaN,37794,492948,290686.0,2.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-23T17:00:00,1,Piemonte,3239,215,3454,33107,36561,-348,933,147203,7679,NaN,NaN,191443,1633219,966374.0,13.0,NaN,NaN
1,2020-12-23T17:00:00,2,Valle d'Aosta,84,4,88,342,430,-26,11,6284,370,NaN,NaN,7084,60760,36709.0,0.0,NaN,NaN
2,2020-12-23T17:00:00,3,Lombardia,4243,536,4779,55460,60239,-905,2153,379750,24610,NaN,NaN,464599,4705264,2556373.0,31.0,NaN,NaN
3,2020-12-23T17:00:00,4,Trentino-Alto Adige,516,66,582,10298,10880,-478,288,36388,1577,0.0,0.0,48845,785198,308427.0,3.0,0,NaN
4,2020-12-23T17:00:00,5,Veneto,2660,350,3010,100316,103326,748,3357,116872,5747,NaN,NaN,225945,3165214,1177012.0,36.0,NaN,NaN
5,2020-12-23T17:00:00,6,Friuli Venezia Giulia,595,56,651,11624,12275,-941,512,32110,1499,NaN,NaN,45884,887279,330123.0,6.0,"Su indicazione del Ministero della Salute, il ...",NaN
6,2020-12-23T17:00:00,7,Liguria,689,64,753,5424,6177,-291,228,49202,2819,NaN,NaN,58198,684768,315604.0,5.0,I 139001 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-23T17:00:00,8,Emilia-Romagna,2803,207,3010,55015,58025,-1721,1129,94188,7259,NaN,NaN,159472,2469690,1194299.0,21.0,NaN,NaN
8,2020-12-23T17:00:00,9,Toscana,903,166,1069,10288,11357,-480,435,102105,3517,NaN,NaN,116979,1818397,1039802.0,7.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-23T17:00:00,10,Umbria,239,40,279,3269,3548,-181,157,23421,584,NaN,NaN,27553,484909,246207.0,1.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-22T17:00:00,1,Piemonte,3331,228,3559,33350,36909,-1048,...,145980,7621,NaN,NaN,190510,1624273,963280.0,7.0,NaN,NaN
1,1,2020-12-22T17:00:00,2,Valle d'Aosta,90,5,95,361,456,-18,...,6249,368,NaN,NaN,7073,60537,36564.0,1.0,NaN,NaN
2,2,2020-12-22T17:00:00,3,Lombardia,4290,540,4830,56314,61144,-170,...,376790,24512,NaN,NaN,462446,4672338,2546141.0,23.0,NaN,NaN
3,19,2020-12-22T17:00:00,4,Trentino-Alto Adige,540,72,612,10746,11358,-61,...,35633,1566,0.0,0.0,48557,779611,307315.0,5.0,0,NaN
4,3,2020-12-22T17:00:00,5,Veneto,2672,349,3021,99557,102578,1104,...,114379,5631,NaN,NaN,222588,3144621,1171789.0,40.0,NaN,NaN
5,4,2020-12-22T17:00:00,6,Friuli Venezia Giulia,598,57,655,12561,13216,-31,...,31291,1477,NaN,NaN,45984,880631,328586.0,7.0,NaN,NaN
6,5,2020-12-22T17:00:00,7,Liguria,700,65,765,5703,6468,-120,...,48704,2798,NaN,NaN,57970,680832,314620.0,1.0,I 135297 test antigenici rapidi sono riferiti ...,NaN
7,6,2020-12-22T17:00:00,8,Emilia-Romagna,2891,210,3101,56645,59746,-2308,...,91411,7188,NaN,NaN,158345,2451777,1189534.0,24.0,NaN,NaN
8,7,2020-12-22T17:00:00,9,Toscana,941,175,1116,10721,11837,-559,...,101212,3495,NaN,NaN,116544,1806133,1035769.0,3.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-22T17:00:00,10,Umbria,244,44,288,3441,3729,-124,...,23089,578,NaN,NaN,27396,481344,245205.0,5.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-23T17:00:00,1,Piemonte,3239,215,3454,33107,36561,-348,933,...,7679,NaN,NaN,191443,1633219,966374.0,13.0,NaN,NaN,4341375
1,2020-12-23T17:00:00,2,Valle d'Aosta,84,4,88,342,430,-26,11,...,370,NaN,NaN,7084,60760,36709.0,0.0,NaN,NaN,125501
2,2020-12-23T17:00:00,3,Lombardia,4243,536,4779,55460,60239,-905,2153,...,24610,NaN,NaN,464599,4705264,2556373.0,31.0,NaN,NaN,10103969
3,2020-12-23T17:00:00,4,Trentino-Alto Adige,516,66,582,10298,10880,-478,288,...,1577,0.0,0.0,48845,785198,308427.0,3.0,0,NaN,1074819
4,2020-12-23T17:00:00,5,Veneto,2660,350,3010,100316,103326,748,3357,...,5747,NaN,NaN,225945,3165214,1177012.0,36.0,NaN,NaN,4907704
5,2020-12-23T17:00:00,6,Friuli Venezia Giulia,595,56,651,11624,12275,-941,512,...,1499,NaN,NaN,45884,887279,330123.0,6.0,"Su indicazione del Ministero della Salute, il ...",NaN,1211357
6,2020-12-23T17:00:00,7,Liguria,689,64,753,5424,6177,-291,228,...,2819,NaN,NaN,58198,684768,315604.0,5.0,I 139001 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-23T17:00:00,8,Emilia-Romagna,2803,207,3010,55015,58025,-1721,1129,...,7259,NaN,NaN,159472,2469690,1194299.0,21.0,NaN,NaN,4467118
8,2020-12-23T17:00:00,9,Toscana,903,166,1069,10288,11357,-480,435,...,3517,NaN,NaN,116979,1818397,1039802.0,7.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-23T17:00:00,10,Umbria,239,40,279,3269,3548,-181,157,...,584,NaN,NaN,27553,484909,246207.0,1.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-23T17:00:00,1,Piemonte,3239,215,3454,33107,36561,-348,933,...,NaN,NaN,191443,1633219,966374.0,13.0,NaN,NaN,4341375,575
1,2020-12-23T17:00:00,2,Valle d'Aosta,84,4,88,342,430,-26,11,...,NaN,NaN,7084,60760,36709.0,0.0,NaN,NaN,125501,20
2,2020-12-23T17:00:00,3,Lombardia,4243,536,4779,55460,60239,-905,2153,...,NaN,NaN,464599,4705264,2556373.0,31.0,NaN,NaN,10103969,1036
3,2020-12-23T17:00:00,4,Trentino-Alto Adige,516,66,582,10298,10880,-478,288,...,0.0,0.0,48845,785198,308427.0,3.0,0,NaN,1074819,106
4,2020-12-23T17:00:00,5,Veneto,2660,350,3010,100316,103326,748,3357,...,NaN,NaN,225945,3165214,1177012.0,36.0,NaN,NaN,4907704,1016
5,2020-12-23T17:00:00,6,Friuli Venezia Giulia,595,56,651,11624,12275,-941,512,...,NaN,NaN,45884,887279,330123.0,6.0,"Su indicazione del Ministero della Salute, il ...",NaN,1211357,180
6,2020-12-23T17:00:00,7,Liguria,689,64,753,5424,6177,-291,228,...,NaN,NaN,58198,684768,315604.0,5.0,I 139001 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-23T17:00:00,8,Emilia-Romagna,2803,207,3010,55015,58025,-1721,1129,...,NaN,NaN,159472,2469690,1194299.0,21.0,NaN,NaN,4467118,634
8,2020-12-23T17:00:00,9,Toscana,903,166,1069,10288,11357,-480,435,...,NaN,NaN,116979,1818397,1039802.0,7.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-23T17:00:00,10,Umbria,239,40,279,3269,3548,-181,157,...,NaN,NaN,27553,484909,246207.0,1.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-23T17:00:00,5,Veneto,2660,350,3010,100316,103326,748,3357,...,34.45,759,275,20593,5223.0,116,2493,16.30,0.06840,4.60
1,2020-12-23T17:00:00,3,Lombardia,4243,536,4779,55460,60239,-905,2153,...,51.74,-854,-125,32926,10232.0,98,2960,6.54,0.02131,4.60
2,2020-12-23T17:00:00,8,Emilia-Romagna,2803,207,3010,55015,58025,-1721,1129,...,32.65,-1630,-33,17913,4765.0,71,2777,6.30,0.02527,3.57
3,2020-12-23T17:00:00,15,Campania,1513,107,1620,78327,79947,-1312,1067,...,21.19,-1285,276,18426,4672.0,19,2360,5.79,0.01844,3.16
4,2020-12-23T17:00:00,12,Lazio,2740,286,3026,72676,75702,-247,946,...,33.77,-261,-342,4392,5911.0,44,1149,21.54,0.01613,2.60
5,2020-12-23T17:00:00,16,Puglia,1417,172,1589,51283,52872,-420,942,...,46.61,-421,66,10492,4718.0,20,1342,8.98,0.02350,2.08
6,2020-12-23T17:00:00,1,Piemonte,3239,215,3454,33107,36561,-348,933,...,37.39,-243,391,8946,3094.0,58,1223,10.43,0.02149,4.41
7,2020-12-23T17:00:00,19,Sicilia,1028,176,1204,32410,33614,122,932,...,29.93,153,38,9264,5809.0,10,800,10.06,0.01876,1.75
8,2020-12-23T17:00:00,6,Friuli Venezia Giulia,595,56,651,11624,12275,-941,512,...,31.11,-937,-37,6648,1537.0,22,819,7.70,0.04227,3.79
9,2020-12-23T17:00:00,11,Marche,411,61,472,9087,9559,62,498,...,42.66,72,135,6090,3992.0,4,432,8.18,0.03280,2.52


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,66,40,106,62.26
1,Lombardia,536,500,1036,51.74
2,Puglia,172,197,369,46.61
3,Marche,61,82,143,42.66
4,Piemonte,215,360,575,37.39
5,Veneto,350,666,1016,34.45
6,Umbria,40,77,117,34.19
7,Lazio,286,561,847,33.77
8,Emilia-Romagna,207,427,634,32.65
9,Toscana,166,357,523,31.74


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,2020-12-21T17:00:00,22,P.A. Trento,382,49,431,1641,2072,-320,59,17227,855,NaN,NaN,20154,430282,148116.0,2.0,NaN,"Dei 59 casi odierni, 16 sono molecolari con pr..."
604,2020-12-22T17:00:00,21,P.A. Bolzano,176,23,199,9156,9355,8,102,18214,698,NaN,NaN,28267,347911,158919.0,2.0,NaN,NaN
605,2020-12-22T17:00:00,22,P.A. Trento,364,49,413,1590,2003,-69,136,17419,868,NaN,NaN,20290,431700,148396.0,3.0,NaN,"Dei 136 casi odierni, 79 sono molecolari con p..."
606,2020-12-23T17:00:00,21,P.A. Bolzano,170,19,189,9025,9214,-141,131,18485,699,NaN,NaN,28398,350170,159457.0,0.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,2020-12-19T17:00:00,4,Trentino-Alto Adige,553,73,626,12547,13173,-430,378,...,12.0,5509.0,1059.0,-430.0,11.66,4.75,6.86,35.69,0.00,-3.16
300,2020-12-20T17:00:00,4,Trentino-Alto Adige,551,69,620,12646,13266,93,354,...,6.0,3377.0,741.0,99.0,11.13,4.67,10.48,47.77,-5.48,0.71
301,2020-12-21T17:00:00,4,Trentino-Alto Adige,561,71,632,10787,11419,-1847,102,...,15.0,1717.0,401.0,-1859.0,11.23,5.53,5.94,25.44,2.90,-13.92
302,2020-12-22T17:00:00,4,Trentino-Alto Adige,540,72,612,10746,11358,-61,238,...,19.0,2353.0,652.0,-41.0,11.76,5.39,10.11,36.50,1.41,-0.53


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
